In [ ]:
#from flask import Flask, render_template, request
#from tensorflow.keras.models import load_model
#from PIL import Image
#import numpy as np


In [ ]:
# Descargando el modelo guardado en drive
from google.colab import drive
drive.mount('/content/drive')

#!cp "/content/drive/My Drive/neumonia.keras" "/content"
!cp "/content/drive/My Drive/inceptionresnetv2_neumonia10_93_25.keras" "/content"


!unzip -uq "/content/drive/My Drive/templates.zip" -d "/content"

Mounted at /content/drive


In [ ]:
!pip install -q ngrok

In [ ]:
!pip install Flask pyngrok

In [ ]:
!pip install Pillow

In [ ]:
import os

nombre_archivo = 'inceptionresnetv2_neumonia10_93_25.keras'  # Reemplaza con el nombre de tu archivo
tamaño_en_bytes = os.path.getsize(nombre_archivo)

print(f"El tamaño del archivo '{nombre_archivo}' es: {tamaño_en_bytes} bytes")

El tamaño del archivo 'inceptionresnetv2_neumonia10_93_25.keras' es: 658894529 bytes


In [ ]:
from tensorflow.keras.models import load_model
# Cargar el modelo
#model = load_model('neumonia.keras')/content/inceptionresnetv2_neumonia10_93_25.keras
model = load_model('inceptionresnetv2_neumonia10_93_25.keras')


In [ ]:
!ngrok authtoken 2kyV6tIpiynlDrKrDR4wGZoTHGp_7eE67e1ZXVwz3aG3jQ5rP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#!ngrok http 5000 &

In [ ]:
#!nohup ngrok http 5000 &

nohup: appending output to 'nohup.out'


In [ ]:
#Para revisar puerto en uso
!lsof -i :5000

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 11806 root   48u  IPv4 428943      0t0  TCP localhost:5000 (LISTEN)


In [ ]:
!kill 11806

In [ ]:
#!ps aux | grep ngrok

root        7934  0.0  0.0   7376  3420 ?        S    10:59   0:00 /bin/bash -c ps aux | grep ngrok
root        7936  0.0  0.0   6484  2260 ?        S    10:59   0:00 grep ngrok


In [ ]:
# Completo funcionando
import logging
logging.basicConfig(level=logging.INFO)

!pip install Flask pyngrok

from flask import Flask, render_template, request
import threading
from pyngrok import ngrok
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

from tensorflow.keras.models import load_model
# Cargar el modelo
model = load_model('inceptionresnetv2_neumonia10_93_25.keras')

app = Flask(__name__)

# Variable global para controlar la ejecución del hilo de Flask
running = True

#Lo pongo en modo depuración
app.config['DEBUG'] = True
app.config['PROPAGATE_EXCEPTIONS'] = True

# Preprocesar la imagen
def preprocess_image(img_path):
    #img = Image.open(img_path)
    #img = img.resize((299, 299))  # Ajustar al tamaño de entrada del modelo
    #img_array = np.array(img)
    #img_array = img_array / 255.0  # Normalizar
    #img_array = np.expand_dims(img_array, axis=0)

    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = x / 255.0  # Normalización
    x = np.expand_dims(x, axis=0)

    return x


@app.route('/')
def index():
  logging.info("Entrando a la función index")

  try:
      return  render_template('index.html') #'¡Hola desde mi página web en Colab!'#
  except Exception as e:
      return f"Error: {str(e)}"
  logging.info("Saliendo de la función index")

print(app.root_path)

# Ruta para procesar la imagen subida
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    predictions = None  # Inicializa la variable con un valor predeterminado

    if request.method == 'POST':
        file = request.files['file']
        img_path = 'temp.jpg'  # Ruta temporal para guardar la imagen
        file.save(img_path)
        img = preprocess_image(img_path)

        # Realiza la predicción
        predictions = model.predict(img)

# Imprime las predicciones
#print('Predicted:', decode_predictions(predictions, top=3)[0])
# Imprime la predicción (sin decode_predictions)
# Asumiendo que un valor > 0.5 indica neumonía

        print(predictions[0][0])
        if predictions[0][0] > 0.5:
            result = 'La imagen sugiere la presencia de neumonía.'
        else:
            result = 'La imagen no sugiere la presencia de neumonía.'
        #return render_template('result.html', result=result)

        # Verifica si prediction tiene un valor antes de acceder a sus elementos
        if predictions is not None:
            result += str(predictions[0][0])
            return render_template('result.html', result=result)
        else:
            return render_template('result.html', result="No se pudo realizar la predicción.")


def run_flask():
    global running
    app.run(debug=True, use_reloader=False)  # Desactiva el reloader para evitar conflictos con el hilo
    running = False

# Crea un hilo para ejecutar Flask en segundo plano
thread = threading.Thread(target=run_flask)
thread.start()

# List and kill any existing ngrok processes
!pkill -f "ngrok"

# Abre un túnel a través de ngrok al puerto 5000 (el puerto predeterminado de Flask)
public_url = ngrok.connect(5000)
print(public_url)

/content
 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


NgrokTunnel: "https://4edc-35-231-112-156.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!pip install --upgrade --force-reinstall keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.4 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.7 MB/s eta